## Version Description
In this version, we deploy a Hybrid PINN to predict two correction factors, i.e the correction factor for both the MTR ( `MTR-600` ) and its corresponding duct part (`TZ-6_zmix`) is predicted by the neural network.

In [149]:
from IPython.core.display import display, HTML,display_html
display(HTML("<style>.container { width:95% !important; }</style>"))

In [150]:
# Source: https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [151]:
# import required libraries
import random
import pandas as pd
import numpy as np

from functools import reduce
import sys

#Set some numpy print options for displaying numpy arrays to fit maximum width of cell
np.set_printoptions(precision=3, edgeitems=30, linewidth=1000,formatter=dict(float=lambda x: "%.3g" % x))

# Disable Warnings for chained assignments Eg:Setting with Copy Warning
pd.options.mode.chained_assignment = None

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_file,output_notebook
output_notebook() # Set to output the plot in the notebook

Loading BokehJS ...

In [152]:
# import LTR and corresponding Restrictor dataframe
Restrictor_data = pd.read_csv('D:/OneDrive/Documents/Airbus/GitHub/ML_A350_Restrictors/TZ6/data/Zone6_RestrictorData_LTR.csv')
LTR_data = pd.read_csv('D:/OneDrive/Documents/Airbus/GitHub/ML_A350_Restrictors/TZ6/data/Zone6_LTRData.csv')
obs_HOVs = pd.read_csv('../data/LTR_dataset.csv', usecols = ['HoV'])[['HoV']]
LTR_data[['AMBP','MIXP']] *= 100 # Convert hPa to Pa
zero_MIXT_hoV_idx = LTR_data.index[LTR_data['MIXT'] == 20].tolist() # Points where originally, MIXT == 0 and substituted with 20° C
LTR_data[['MIXT','AMBT']] += 273.15 # Convert Celcius to Kelvin
print('Total LTR points:',len(LTR_data))

## Calculate Density = (0.5* Pmix + Pamb) / (R * Tmix) where R = 287 is the Gas constant in Ideal Gas Law
idx = LTR_data.columns.get_loc("AMBT")
LTR_data.insert(loc=(idx+1), column='Density(kg/m³)', value=np.round((0.5 * LTR_data['MIXP'] + LTR_data['AMBP'] ) / (LTR_data['MIXT']*287),4))
target_df = reduce(lambda left,right: pd.merge(left,right,left_on='HoV',right_on='HoV'), [LTR_data,Restrictor_data])
target_df['HoV'] = obs_HOVs.values
target_df.head(3)

Total LTR points: 34


,HoV,CAOLH_C66-C68,CAOLH_C68-C70,CAOLH_C70-C72,CAOLH_C72-C74,CAOLH_C74-C76,CAOLH_C76-C78,CAORH_C66-C68,CAORH_C68-C70,CAORH_C70-C72,...,R632_HS1,R633_HS1,R634_HS1,R635_HS1,R636_HS1,R637_HS1,R638_HS1,R639_HS1,R640_HS1,R641_HS1
0,A1,36.071000,38.802000,36.796000,36.794000,39.100000,38.318000,37.055000,39.596000,38.333000,...,41,41,41,41,41,41,41,41,41,41
1,A2,36.061000,38.697000,37.156000,36.738000,38.957000,38.326000,36.501000,40.797000,39.284000,...,41,41,41,41,41,41,41,41,41,41
2,A3,37.504234,40.911735,38.412733,37.859887,36.585486,38.504564,36.161971,37.068225,37.893405,...,41,41,41,41,41,41,41,41,41,41


In [153]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 0 to 33
Data columns (total 58 columns):
HoV               34 non-null object
CAOLH_C66-C68     34 non-null float64
CAOLH_C68-C70     34 non-null float64
CAOLH_C70-C72     34 non-null float64
CAOLH_C72-C74     34 non-null float64
CAOLH_C74-C76     34 non-null float64
CAOLH_C76-C78     34 non-null float64
CAORH_C66-C68     34 non-null float64
CAORH_C68-C70     34 non-null float64
CAORH_C70-C72     34 non-null float64
CAORH_C72-C74     34 non-null float64
CAORH_C74-C76     34 non-null float64
CAORH_C76-C78     34 non-null float64
LAOLH_C66-C68     34 non-null float64
LAOLH_C68-C70     34 non-null float64
LAOLH_C70-C72     34 non-null float64
LAOLH_C72-C74     34 non-null float64
LAOLH_C74-C76     34 non-null float64
LAOLH_C76-C78     34 non-null float64
LAORH_C66-C68     34 non-null float64
LAORH_C68-C70     34 non-null float64
LAORH_C70-C72     34 non-null float64
LAORH_C72-C74     34 non-null float64
LAORH_C74-C76     34 non

In [154]:
MIXP = [1800, 2600, 3400]
# Define Restrictor Labels
MTR = 'R600_HD'
CLRS = ['R610_HS1','R611_HS1','R612_HS1','R613_HS1']
CAORS = ['R620_HS1','R620_HS2','R620_HS3','R621_HS1','R621_HS2','R621_HS3']
MHRS = CLRS + CAORS
LAORS = ['R630_HS1','R631_HS1','R632_HS1','R633_HS1','R634_HS1','R635_HS1','R636_HS1','R637_HS1','R638_HS1','R639_HS1','R640_HS1','R641_HS1']

In [155]:
# Duct Areas
MTR_DuctArea = 0.03464 # in sq.m
MHR_Duct_Areas = {'R610_HS1':0.01767,'R611_HS1':0.01767,'R612_HS1':0.02461,'R613_HS1':0.02461,
                 'R620_HS1':0.01090,'R620_HS2':0.00470,'R620_HS3':0.01085,'R621_HS1':0.01085,'R621_HS2':0.00470,'R621_HS3':0.01090}

## Define Initial Correction factor values from CFD
# Correction Factor for Duct Part `TZ6_zmix:
TZ6_zmix_cf = 1 # Start with initial c_f =1 

#3D Correction Factor for MTR
MTR_cf = 1.1664876228437


#3D Correction Factors for MultiHole Restrictors
MHR_cf = {'R610_HS1':0.740773983383846,'R611_HS1':0.740205245723713,'R612_HS1':0.83879652680329,'R613_HS1':0.81775586281569,
          'R620_HS1':0.757121843184036,'R620_HS2':0.773919779037984,'R620_HS3':0.789028593182055,
          'R621_HS1':0.714811824527744,'R621_HS2':0.866271631648556,'R621_HS3':0.822447592395462}

### Calculate Loss-Coefficient $\zeta$
**For Single Hole Restrictor:**<br>

    f0_f1 = A_Circular(Hole_Diameter) / Area_Overall
    l_cross = Thickness / Hole_Diameter
    Zeta_dash = 0.13 + 0.34 * 10 ^ -(3.4 * l_cross + 88.4 * l_cross ^ 2.3)
    Zeta_Single_Hole_Thick_Chamfered = ((1 - f0_f1 + (Zeta_dash ^ 0.5) * (1 - f0_f1) ^ 0.375) ^ 2) * f0_f1 ^ -2

In [156]:
def SHR_Zeta_3D(n_holes,hole_dia,MTR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Single Hole Retrictors
    '''
    MTR_New_Area = n_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MTR_New_Area/MTR_DuctArea
    l_cross = 1/hole_dia
    zeta_dash = 0.13 + 0.34 * 10**(-(3.4 * l_cross + 88.4 * l_cross**2.3))
    zeta_SHR_1D = ((1 - f0_f1 + (zeta_dash**0.5) * (1 - f0_f1)**0.375)**2) * f0_f1**(-2) # 1D Zeta
    zeta_SHR_3D = zeta_SHR_1D * cf # Zeta with 3D Correction Factor    
    return MTR_New_Area,zeta_SHR_3D

In [157]:
target_df[MTR+'_Area'],target_df[MTR+'_Zeta3D'] = zip(*[SHR_Zeta_3D(1,dia,MTR_DuctArea,MTR_cf) for dia in target_df[MTR]])
target_df[[MTR,'R600_HD_Area','R600_HD_Zeta3D']].head()

,R600_HD,R600_HD_Area,R600_HD_Zeta3D
0,148,0.017203,4.949375
1,149,0.017437,4.730821
2,152,0.018146,4.127180
3,154,0.018627,3.764504
4,148,0.017203,4.949375


**For Multi Hole Restrictors:**<br>

    Area_Free = Number_of_Holes * A_Circular(Hole_Diameter)
    f0_f1 = Area_Free / Area_Overall
    l_cross = Thickness / Hole_Diameter
    phi = 0.25 + (0.535 * l_cross ^ 8) / (0.05 + l_cross ^ 7)
    tau = (2.4 - l_cross) * 10 ^ (-phi)
    Zeta_Multi_Hole = (0.5 * (1 - f0_f1) ^ 0.75 + tau * (1 - f0_f1) ^ 1.375 + (1 - f0_f1) ^ 2 + 0.02 * l_cross) / f0_f1 ^ 2

In [158]:
def MHR_Zeta_3D(nr_holes,hole_dia,MHR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Multi Hole Restrictors
    '''
    MHR_New_Area = nr_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MHR_New_Area/MHR_DuctArea
    l_cross = (0.00144*1000)/hole_dia
    phi = 0.25 + (0.535 * l_cross**8) / (0.05 + l_cross**7)
    tau = (2.4 - l_cross) * 10**(-phi)
    zeta_MHR_1D = (0.5 * (1 - f0_f1)**0.75 + tau * (1 - f0_f1)**1.375 + (1 - f0_f1)**2 + 0.02 * l_cross) / f0_f1**2 # 1D Zeta     
    zeta_MHR_3D = zeta_MHR_1D * cf # Zeta with 3D Correction Factor    
    return MHR_New_Area,zeta_MHR_3D

In [159]:
# Calculate Zeta for Multi-Hole Restrictors
for clr in CLRS:
    if (clr == 'R610_HS1') | (clr == 'R611_HS1'):
        hole_dia = 8
    else:
        hole_dia = 10
    MHR_nr_holes = target_df[clr].values
    target_df[clr+'_Area'], target_df[clr+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele,hole_dia,MHR_Duct_Areas[clr],MHR_cf[clr]) for ele in MHR_nr_holes])
    
for caor in CAORS:
    CAOR_nr_holes = target_df[caor].values
    target_df[caor+'_Area'],target_df[caor+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele,8,MHR_Duct_Areas[caor],MHR_cf[caor]) for ele in CAOR_nr_holes])

In [160]:
# LTR Dataframe col index for Restrictor Zeta and Area values
target_df.iloc[:,58:].head()

,R600_HD_Area,R600_HD_Zeta3D,R610_HS1_Area,R610_HS1_Zeta3D,R611_HS1_Area,R611_HS1_Zeta3D,R612_HS1_Area,R612_HS1_Zeta3D,R613_HS1_Area,R613_HS1_Zeta3D,...,R620_HS2_Area,R620_HS2_Zeta3D,R620_HS3_Area,R620_HS3_Zeta3D,R621_HS1_Area,R621_HS1_Zeta3D,R621_HS2_Area,R621_HS2_Zeta3D,R621_HS3_Area,R621_HS3_Zeta3D
0,0.017203,4.949375,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.00377,8.801675,0.00191,6.836716,0.00568,2.95119
1,0.017437,4.730821,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.00377,8.801675,0.00191,6.836716,0.00568,2.95119
2,0.018146,4.127180,0.006585,7.506119,0.006836,6.741078,0.008954,9.156844,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.00377,8.801675,0.00191,6.836716,0.00568,2.95119
3,0.018627,3.764504,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.00377,8.801675,0.00191,6.836716,0.00568,2.95119
4,0.017203,4.949375,0.007288,5.595443,0.007691,4.756868,0.010210,6.271347,0.010210,6.114035,...,0.00191,6.107864,0.00568,2.786638,0.00377,8.801675,0.00191,6.836716,0.00568,2.95119


### Run FDDN Solver

In [161]:
from FDDN_Lib import fddn_zeta_input, FDDN_Solver, FDDN_output_df_gen, name_variables, zeta_values

In [162]:
print(name_variables)

['TZ-6_zmix', 'MTR-600', 'riser_LAO_RHS', 'riser_LAO_LHS', 'riser_CAO_RHS', 'riser_CAO_LHS', 'R-610', 'R-611', 'R-612', 'R-613', 'LAOR-630', 'LAOR-632', 'LAOR-634', 'LAOR-636', 'LAOR-638', 'LAOR-640', 'LAOR-631', 'LAOR-633', 'LAOR-635', 'LAOR-637', 'LAOR-639', 'LAOR-641', 'CAO-620_fwd', 'CAO-620_mid', 'CAO-620_aft', 'CAO-621_fwd', 'CAO-621_mid', 'CAO-621_aft', 'LAO-630', 'LAO-632', 'LAO-634', 'LAO-636', 'LAO-638', 'LAO-640', 'LAO-631', 'LAO-633', 'LAO-635', 'LAO-637', 'LAO-639', 'LAO-641', 'CAOR-620_fwd', 'CAOR-620_mid', 'CAOR-620_aft', 'CAOR-621_fwd', 'CAOR-621_mid', 'CAOR-621_aft', 'pp2610pdloss', 'pp2611pdloss', 'pp2612pdloss', 'pp2613pdloss', '26120-R620_F', '26120-R620_M', '26120-R620_A', '26121-R621_F', '26121-R621_M', '26121-R621_A']


In [163]:
# Use Initial Zeta value from the CFD model for the 'TZ-6_zmix' duct part
idx = target_df.columns.get_loc("R600_HD_Zeta3D")
target_df.insert(loc=(idx-1), column='TZ6_zmix_Zeta3D', value=float(zeta_values[0]))

In [164]:
# Gather col_names and their corresponding column indices
zeta_col_names = [col for col in target_df.columns if 'Zeta3D' in col]
zeta_col_names_idx = [target_df.columns.get_loc(col) for col in zeta_col_names if col in target_df]
area_col_names = [col for col in target_df.columns if 'Area' in col]
area_col_names_idx = [target_df.columns.get_loc(col) for col in area_col_names if col in target_df]
pt_var_idx = [target_df.columns.get_loc(col) for col in ['MIXP','AMBP','AMBT'] if col in target_df]
req_flow_rates_idx = [target_df.columns.get_loc(col) for col in ['TZ6_Flow'] if col in target_df]
# print(zeta_col_names_idx)
print(zeta_col_names)

['TZ6_zmix_Zeta3D', 'R600_HD_Zeta3D', 'R610_HS1_Zeta3D', 'R611_HS1_Zeta3D', 'R612_HS1_Zeta3D', 'R613_HS1_Zeta3D', 'R620_HS1_Zeta3D', 'R620_HS2_Zeta3D', 'R620_HS3_Zeta3D', 'R621_HS1_Zeta3D', 'R621_HS2_Zeta3D', 'R621_HS3_Zeta3D']


In [165]:
LTR_zeta_df_col_idx = sorted([0] + req_flow_rates_idx + pt_var_idx + zeta_col_names_idx)
target_df.iloc[:,LTR_zeta_df_col_idx].to_csv('data_output/LTR_Zeta_df.csv', index=False)

In [166]:
LTR_zeta_df = target_df[['HoV','MIXP','AMBP','AMBT','TZ6_Flow']+zeta_col_names]
LTR_zeta_df[['HoV','AMBP','AMBT']+zeta_col_names].tail()

,HoV,AMBP,AMBT,TZ6_zmix_Zeta3D,R600_HD_Zeta3D,R610_HS1_Zeta3D,R611_HS1_Zeta3D,R612_HS1_Zeta3D,R613_HS1_Zeta3D,R620_HS1_Zeta3D,R620_HS2_Zeta3D,R620_HS3_Zeta3D,R621_HS1_Zeta3D,R621_HS2_Zeta3D,R621_HS3_Zeta3D
29,S3,101105.3000,297.285667,0.712987,4.521189,7.506119,6.741078,7.920333,7.721657,9.440922,6.107864,2.786638,8.801675,6.836716,2.95119
30,S4,102356.1438,294.425869,0.712987,4.521189,7.506119,6.741078,7.920333,6.862828,9.440922,6.107864,2.786638,8.801675,6.836716,2.95119
31,S5,102286.7009,294.434538,0.712987,2.839767,7.506119,5.591147,7.920333,6.862828,9.440922,6.107864,2.786638,8.801675,6.836716,2.95119
32,T1,100112.6000,297.133500,0.712987,4.949375,7.506119,6.741078,7.920333,7.721657,9.440922,6.107864,2.786638,8.801675,6.836716,2.95119
33,T2,101099.0000,296.278833,0.712987,4.949375,7.506119,6.741078,7.920333,7.721657,9.440922,6.107864,2.786638,8.801675,6.836716,2.95119


In [167]:
final_df = target_df[['HoV','MIXP','Density(kg/m³)','R600_HD','TZ6_zmix_Zeta3D','R600_HD_Zeta3D','TZ6_Flow']]
final_df.reset_index(drop=True,inplace = True)
# final_df[['MIXP','Density(kg/m³)','R600_HD','TZ6_zmix_Zeta3D','R600_HD_Zeta3D','TZ6_Flow']].head()

### Training data Preparation

In [168]:
from Hybrid_PINN_v2 import NormbyMax, DeNormbyMax, PINN

In [169]:
input_features = ['MIXP','Density(kg/m³)','TZ6_zmix_Zeta3D','R600_HD_Zeta3D','TZ6_Flow']
features_max, df_rescaled = NormbyMax(final_df,input_features)
print('Max value of all features:',features_max)

V_max_org = features_max['TZ6_Flow']
print('Maximum TZ6 Flow:',V_max_org)

Max value of all features: {'MIXP': 2632.9689, 'Density(kg/m³)': 1.238, 'TZ6_zmix_Zeta3D': 0.712986788872249, 'R600_HD_Zeta3D': 5.4148802645163, 'TZ6_Flow': 764.0123719999998}
Maximum TZ6 Flow: 764.0123719999998


In [170]:
# Display original and Scaled DataFrame Side-by-side
# print( '{:<60s} {:<20s}'.format('Original Input Dataframe:','Rescaled Input Dataframe:') )
# display_side_by_side(final_df[['HoV']+input_features],df_rescaled)

In [171]:
df_rescaled_inv = DeNormbyMax(df_rescaled,features_max,input_features)

In [172]:
train_data = df_rescaled
# train_data = df_rescaled.iloc[[7,15]]
print("Number of training samples:", len(train_data))
train_data

Number of training samples: 34


,MIXP,Density(kg/m³),TZ6_zmix_Zeta3D,R600_HD_Zeta3D,TZ6_Flow
0,0.987478,0.986026,1.0,0.914032,0.966062
1,0.987478,0.987722,1.0,0.873670,0.965658
2,0.989830,0.976575,1.0,0.762192,0.969853
3,0.987440,0.999031,1.0,0.695215,0.958246
4,0.987478,1.000000,1.0,0.914032,0.970963
5,0.987478,0.985299,1.0,0.914032,0.973195
6,0.987478,0.978837,1.0,0.914032,0.968664
7,0.988183,0.967367,1.0,0.728011,0.971982
8,0.985737,0.960420,1.0,0.762192,0.968232
9,0.987478,0.996446,1.0,0.834956,0.968331


In [173]:
# Store indices and values of train data
train_data_idx = train_data.index.values.tolist()
train_R600HD_series = final_df['R600_HD'].iloc[train_data_idx]
X_train = train_data.values

### Predict $c_f$ with Physics Informed Neural Network

In [28]:
# A random seed is used to reproduce the weight initialization values and training performance everytime this notebook is run.
np.random.seed(13)
# Define custom MSE function
MSE = lambda y_hat,y_true: np.mean((y_hat-y_true)**2)

**Train the Neural Network**:

In [29]:
### Set the hyperparameters here ###
iterations = 100
learning_rate = 0.01 ### Important Parameter
hidden_nodes = 3 ### Important Parameter
output_nodes = 2
MTR_epsi = MTR_cf * 0.01
TZ6_zmix_epsi = TZ6_zmix_cf * 0.01

In [30]:
N_i = X_train.shape[1]
network = PINN(N_i, hidden_nodes, output_nodes, learning_rate)

train_loss_hist = []
losses_hist = {'HoV':[],'train_loss':[]}
c_f_hist = {'HoV':[],'MTR_cf_hat':[],'TZ6zmix_cf_hat':[]}

for idx in train_data_idx:    
    ## Stochastic Gradient Descent with mini-batch training
    print('Old_Zeta:',target_df[MTR+'_Zeta3D'].loc[idx])
    for ii in range(iterations):    
        # Go through one record at a time from the training data set
        X_train = train_data[input_features].loc[[idx]].values
        dia_train = np.atleast_1d(train_R600HD_series.loc[idx])    
        HoV,V_hat_FDDN,V_true_LTR,break_flag,MTR_new_cf,TZ6_zmix_new_cf = network.train(X_train,dia_train,[idx],(MTR_epsi,TZ6_zmix_epsi),
                                                                                        MTR_DuctArea,target_df,zeta_col_names,V_max_org)
        
        # Printing out the training progress
        train_loss = MSE(V_hat_FDDN,V_true_LTR) # MSE between FDDN predicted flow rate and true flow rate on UN-scaled original values    
        sys.stdout.write("\rProgress: {:2.1f}".format(100 * ii/float(iterations))+ "% ... Training loss: " + str(round(train_loss,4))[:5])
        print('... Training Iteration:',ii)    
        sys.stdout.flush()
        train_loss_hist.append(train_loss)
        if (break_flag == 1) or (ii == iterations-1): # Early stopping based on custom break flag during training
            print('EARLY STOPPING ACTIVATED - Terminating Neural Network Training')
            print('Train Loss History for {}:'.format(HoV[0]),train_loss_hist)
            losses_hist['HoV'].append(HoV[0])
            losses_hist['train_loss'].append(train_loss_hist)
            c_f_hist['HoV'].append(HoV[0])
            c_f_hist['MTR_cf_hat'].append(MTR_new_cf)
            c_f_hist['TZ6zmix_cf_hat'].append(TZ6_zmix_new_cf)
            train_loss_hist = []
            break

Old_Zeta: 4.949375283472444

NN output (C_f): [[1.14 -0.0976]]
MTR-Cf: 1.1409931844143717 , TZ6-zmix-Cf: -0.09761142606542361

NN output is -ve. Taking |(c_f)| values for both MTR and TZ6_zmix: [[1.14 0.0976]]
MTR-Cf: 1.1409931844143717
|TZ6_zmix-Cf|: 0.09761142606542361
New_Zeta for MTR-600: [4.84]
New_Zeta for TZ6_zmix: [0.0696]
FDDN Solver Output: [[76.2 36.4 115 75.1 39.7 122 12 26 25.9 26 25.4 25.1 11.7 25.6 25.8 25.3 25 25]]
New_Zeta_epsi for MTR-600: [4.89]
New_Zeta_epsi for TZ6_zmix: [-0.00749]
FDDN Solver Output: [[76.3 36.4 115 75.2 39.7 123 12 26 25.9 26 25.4 25.1 11.7 25.6 25.8 25.4 25.1 25.1]]
Row ID: [0]
HoV: ['A1']
FlowRate Difference (LTR - FDDN): [-5.51] l/s
% Diff between flowrates [0.743]
Progress: 0.0% ... Training loss: 30.31... Training Iteration: 0
EARLY STOPPING ACTIVATED - Terminating Neural Network Training
Train Loss History for A1: [30.317672124050258]
Old_Zeta: 4.7308206417914045

NN output (C_f): [[1.12 -0.152]]
MTR-Cf: 1.1228167334473969 , TZ6-zmix-Cf: -0

FDDN Solver Output: [[77 36.8 116 74 39.1 121 12.2 26.2 26.2 26.2 25.7 25.3 11.9 25.8 26 25.6 25.3 25.3]]
New_Zeta_epsi for MTR-600: [4.7]
New_Zeta_epsi for TZ6_zmix: [2.13e-19]
FDDN Solver Output: [[76.9 36.7 116 73.9 39 120 12.1 26.2 26.1 26.2 25.6 25.3 11.8 25.8 26 25.6 25.3 25.3]]
Row ID: [2]
HoV: ['A3']
FlowRate Difference (LTR - FDDN): [-4.56] l/s
% Diff between flowrates [0.613]
Progress: 0.0% ... Training loss: 20.74... Training Iteration: 0
EARLY STOPPING ACTIVATED - Terminating Neural Network Training
Train Loss History for A3: [20.74903276879855]
Old_Zeta: 3.7645035449506787

NN output (C_f): [[1.44 -0.276]]
MTR-Cf: 1.441739665193178 , TZ6-zmix-Cf: -0.275702478011363

NN output is -ve. Taking |(c_f)| values for both MTR and TZ6_zmix: [[1.44 0.276]]
MTR-Cf: 1.441739665193178
|TZ6_zmix-Cf|: 0.275702478011363
New_Zeta for MTR-600: [4.65]
New_Zeta for TZ6_zmix: [5.88e-20]
FDDN Solver Output: [[76 36.3 115 75 39.6 122 12 25.9 25.8 25.9 25.3 25 11.7 25.5 25.7 25.3 25 25]]
New_Zeta


NN output (C_f): [[1.1 -0.523]]
MTR-Cf: 1.0959365204593758 , TZ6-zmix-Cf: -0.5227279793138901

NN output is -ve. Taking |(c_f)| values for both MTR and TZ6_zmix: [[1.1 0.523]]
MTR-Cf: 1.0959365204593758
|TZ6_zmix-Cf|: 0.5227279793138901
New_Zeta for MTR-600: [4.65]
New_Zeta for TZ6_zmix: [2.71e-28]
FDDN Solver Output: [[76.5 36.5 115 75.4 39.8 123 12.1 26 26 26.1 25.5 25.2 11.8 25.6 25.9 25.4 25.1 25.1]]
New_Zeta_epsi for MTR-600: [4.7]
New_Zeta_epsi for TZ6_zmix: [-1.44e-28]
FDDN Solver Output: [[76.3 36.4 115 75.2 39.7 123 12 26 25.9 26 25.4 25.1 11.7 25.6 25.8 25.4 25.1 25.1]]
Row ID: [5]
HoV: ['A6']
FlowRate Difference (LTR - FDDN): [-2.67] l/s
% Diff between flowrates [0.358]
Progress: 1.0% ... Training loss: 7.125... Training Iteration: 1
EARLY STOPPING ACTIVATED - Terminating Neural Network Training
Train Loss History for A6: [143.00648071254003, 7.125908620682836]
Old_Zeta: 4.949375283472444

NN output (C_f): [[1.13 -0.527]]
MTR-Cf: 1.1287434333449018 , TZ6-zmix-Cf: -0.5267296


NN output (C_f): [[1.49 -0.702]]
MTR-Cf: 1.4930418721715184 , TZ6-zmix-Cf: -0.7017709375738066

NN output is -ve. Taking |(c_f)| values for both MTR and TZ6_zmix: [[1.49 0.702]]
MTR-Cf: 1.4930418721715184
|TZ6_zmix-Cf|: 0.7017709375738066
New_Zeta for MTR-600: [5.28]
New_Zeta for TZ6_zmix: [1.77e-32]
FDDN Solver Output: [[74.1 35.3 112 73 38.6 119 11.8 25.5 25.5 25.5 25 24.7 11.4 24.8 25.1 24.6 24.3 24.3]]
New_Zeta_epsi for MTR-600: [5.32]
New_Zeta_epsi for TZ6_zmix: [-1.26e-32]
FDDN Solver Output: [[73.9 35.3 111 72.9 38.5 119 11.8 25.5 25.4 25.5 24.9 24.6 11.4 24.8 25 24.6 24.3 24.3]]
Row ID: [10]
HoV: ['C5']
FlowRate Difference (LTR - FDDN): [2.73] l/s
% Diff between flowrates [0.376]
Progress: 1.0% ... Training loss: 7.466... Training Iteration: 1
EARLY STOPPING ACTIVATED - Terminating Neural Network Training
Train Loss History for C5: [700.5032084821667, 7.466294175162057]
Old_Zeta: 3.7645035449506787

NN output (C_f): [[1.54 -0.76]]
MTR-Cf: 1.5443316922850387 , TZ6-zmix-Cf: -0.7

EARLY STOPPING ACTIVATED - Terminating Neural Network Training
Train Loss History for E2: [4090.527527680384, 22.75900897872221]
Old_Zeta: 5.4148802645163

NN output (C_f): [[1.69 -0.623]]
MTR-Cf: 1.6891924788614008 , TZ6-zmix-Cf: -0.6227103044333142

NN output is -ve. Taking |(c_f)| values for both MTR and TZ6_zmix: [[1.69 0.623]]
MTR-Cf: 1.6891924788614008
|TZ6_zmix-Cf|: 0.6227103044333142
New_Zeta for MTR-600: [7.84]
New_Zeta for TZ6_zmix: [5.27e-34]
FDDN Solver Output: [[66.2 31.6 99.8 65.3 34.5 106 10.5 22.6 22.5 22.6 22.1 21.8 10.2 22.2 22.4 22 21.8 21.8]]
New_Zeta_epsi for MTR-600: [7.9]
New_Zeta_epsi for TZ6_zmix: [-3.33e-34]
FDDN Solver Output: [[66.1 31.5 99.6 65.1 34.4 106 10.4 22.5 22.5 22.5 22 21.7 10.2 22.2 22.4 22 21.7 21.7]]
Row ID: [16]
HoV: ['F1']
FlowRate Difference (LTR - FDDN): [75.3] l/s
Progress: 0.0% ... Training loss: 5665.... Training Iteration: 0

NN output (C_f): [[1.12 -0.796]]
MTR-Cf: 1.1181370026660207 , TZ6-zmix-Cf: -0.7961769182438221

NN output is -ve.


NN output (C_f): [[1.41 -0.851]]
MTR-Cf: 1.4077069899739516 , TZ6-zmix-Cf: -0.8508330548183908

NN output is -ve. Taking |(c_f)| values for both MTR and TZ6_zmix: [[1.41 0.851]]
MTR-Cf: 1.4077069899739516
|TZ6_zmix-Cf|: 0.8508330548183908
New_Zeta for MTR-600: [5.97]
New_Zeta for TZ6_zmix: [1.82e-35]
FDDN Solver Output: [[71.9 34.3 108 70.9 37.5 116 11.4 24.5 24.5 24.5 24 23.7 11.1 24.1 24.3 23.9 23.6 23.6]]
New_Zeta_epsi for MTR-600: [6.02]
New_Zeta_epsi for TZ6_zmix: [-1.57e-35]
FDDN Solver Output: [[71.8 34.2 108 70.8 37.4 115 11.3 24.4 24.4 24.5 23.9 23.6 11.1 24.1 24.3 23.9 23.6 23.6]]
Row ID: [21]
HoV: ['J1']
FlowRate Difference (LTR - FDDN): [5.79] l/s
Progress: 1.0% ... Training loss: 33.56... Training Iteration: 1

NN output (C_f): [[1.4 -0.885]]
MTR-Cf: 1.4014361325652442 , TZ6-zmix-Cf: -0.8851810465326901

NN output is -ve. Taking |(c_f)| values for both MTR and TZ6_zmix: [[1.4 0.885]]
MTR-Cf: 1.4014361325652442
|TZ6_zmix-Cf|: 0.8851810465326901
New_Zeta for MTR-600: [5.95]

FDDN Solver Output: [[73.7 35.2 111 72.4 38.2 118 11.7 25.3 25.2 25.3 24.8 24.4 11.4 24.9 25.1 24.7 24.4 24.4]]
New_Zeta_epsi for MTR-600: [5.15]
New_Zeta_epsi for TZ6_zmix: [-2.01e-34]
FDDN Solver Output: [[73.6 35.1 111 72.2 38.2 118 11.7 25.2 25.2 25.2 24.7 24.4 11.4 24.8 25 24.6 24.3 24.3]]
Row ID: [25]
HoV: ['Q1']
FlowRate Difference (LTR - FDDN): [4.78] l/s
% Diff between flowrates [0.661]
Progress: 2.0% ... Training loss: 22.82... Training Iteration: 2
EARLY STOPPING ACTIVATED - Terminating Neural Network Training
Train Loss History for Q1: [735.9261206517881, 104.5416809161247, 22.8275966602428]
Old_Zeta: 4.949375283472444

NN output (C_f): [[1.18 -1.3]]
MTR-Cf: 1.1789712439850055 , TZ6-zmix-Cf: -1.3002411593252754

NN output is -ve. Taking |(c_f)| values for both MTR and TZ6_zmix: [[1.18 1.3]]
MTR-Cf: 1.1789712439850055
|TZ6_zmix-Cf|: 1.3002411593252754
New_Zeta for MTR-600: [5]
New_Zeta for TZ6_zmix: [-2.61e-34]
FDDN Solver Output: [[76.1 36.3 115 75 39.6 122 12 25.9 25.9 25.

FDDN Solver Output: [[76.6 36.5 115 74.2 39.2 121 11.9 25.6 25.6 25.7 25.1 24.8 11.6 25.2 25.5 25 24.7 24.7]]
New_Zeta_epsi for MTR-600: [4.88]
New_Zeta_epsi for TZ6_zmix: [-1.3e-32]
FDDN Solver Output: [[76.4 36.5 115 74.1 39.1 121 11.9 25.6 25.5 25.6 25 24.7 11.6 25.2 25.4 25 24.7 24.7]]
Row ID: [30]
HoV: ['S4']
FlowRate Difference (LTR - FDDN): [-3.92] l/s
% Diff between flowrates [0.533]
Progress: 1.0% ... Training loss: 15.38... Training Iteration: 1
EARLY STOPPING ACTIVATED - Terminating Neural Network Training
Train Loss History for S4: [109.90347840875435, 15.386263222167731]
Old_Zeta: 2.8397674398682766

NN output (C_f): [[1.44 -1.37]]
MTR-Cf: 1.4357057550509809 , TZ6-zmix-Cf: -1.3722432353371528

NN output is -ve. Taking |(c_f)| values for both MTR and TZ6_zmix: [[1.44 1.37]]
MTR-Cf: 1.4357057550509809
|TZ6_zmix-Cf|: 1.3722432353371528
New_Zeta for MTR-600: [3.5]
New_Zeta for TZ6_zmix: [-1.78e-32]
FDDN Solver Output: [[82.6 39.4 125 80.1 42.3 130 13 28.1 28.1 28.1 27.5 27.2 1

EARLY STOPPING ACTIVATED - Terminating Neural Network Training
Train Loss History for T2: [818.6902450323936, 115.85493920227923, 23.4601946304504]


In [31]:
cf_df = pd.DataFrame(data=c_f_hist)
losses = pd.DataFrame(data=losses_hist)

In [32]:
cf_df['MaxEpochs'] = None # Initialize Empty Column for storing epochs trained

for i in range(len(losses['HoV'])):
    nr_epochs_trained = len(losses['train_loss'][i])
    cf_df['MaxEpochs'].loc[i] = nr_epochs_trained

cf_df

,HoV,MTR_cf_hat,TZ6zmix_cf_hat,MaxEpochs
0,A1,1.140993,0.097611,1
1,A2,1.223367,0.213955,10
2,A3,1.317834,0.253589,1
3,A4,1.552047,0.311292,3
4,A5,1.159198,0.467648,6
5,A6,1.095937,0.522728,2
6,C1,1.128743,0.526730,1
7,C2,1.509169,0.555727,3
8,C3,1.473994,0.620002,2
9,C4,1.237740,0.758507,2


In [33]:
print('Max no. of epochs for:',cf_df['HoV'].iloc[[cf_df.MaxEpochs.idxmax()]].values)
cf_df.to_csv('data_output/NN_2CF_Output.csv',index=False)
cf_df.iloc[[cf_df.MaxEpochs.idxmax()]]

Max no. of epochs for: ['A2']


,HoV,MTR_cf_hat,TZ6zmix_cf_hat,MaxEpochs
1,A2,1.223367,0.213955,10


**Plot Training Loss**

In [34]:
from bokeh.palettes import Category20,Colorblind,Spectral,Set2,YlGnBu,RdPu
temp_list = []
bokeh_palettes = [Colorblind,YlGnBu,RdPu,Set2]
for palette in bokeh_palettes:
    for key in palette.keys():
        temp_list.append(palette[key])
    
color_palette = [y for x in temp_list for y in x]

In [35]:
num_lines = len(losses.HoV) # no. of lines to draw
colors = color_palette[0:num_lines]
labels = losses.HoV.values.tolist()

p = figure(width=1100)
for i in range(num_lines):
    x = list(range(1,len(losses['train_loss'][i])+1))
    y = losses['train_loss'][i]
    p.line(x, y, line_width=2, color=colors[i], alpha=0.8, legend='Train Loss for {}'.format(labels[i]))
p.yaxis.axis_label = "Train Loss"
p.xaxis.axis_label = "Epochs"
p.legend.click_policy="hide"
show(p)

### Observations:
In general the performance of the Hybrid PINN architecture seems to be satisfactory and the final `c_f`s predicted are physically consistent. Some drawbacks that were observed are:
* The early stopping criteria is specified in the `Hybrid_PINN_v2.py` script which uses a percentage difference (set to 0.75%) between the LTR and FDDN Flowrate. This % diff value is higher than the value used in the `Hybrid_PINN_v1.py` because in this version we are predicting two correction factors.